Sustainability team project BDM

In [73]:
from pyspark.sql import SparkSession, column
import os 
import sys 
import json 

os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

In [74]:
# Creazione contesto
spark = (
    SparkSession
        .builder
        .appName('test.1_pal')
        .getOrCreate()
)

spark

In [75]:
poll_dir = r"C:\Users\palaz\OneDrive\Desktop\University\UNIPA 2.0\I ANNO\II semestre\Data Processing\BDM - Big Data Management\Progetto 2\BigDataProject\data\poll_data.csv"

df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("mode", "FAILFAST")
    .option("nullValue", "")    #Replace any null data field with quotes
    .load(poll_dir)
) 

#Eliminazione colonne non necessarie: colonne informazioni generali + colonna 81 
df = (
    df.drop("Informazioni cronologiche")
    .drop(
        "Ho letto e accettato l'informativa e confermo inoltre di avere più di 18 anni"
    )
    .drop("Quanti anni hai?")
    .drop("Genere")
    .drop("Da quante persone è composto il tuo nucleo familiare?")
    .drop("Occupazione")
    .drop("Quanto è grande la tua azienda?")
    .drop("Da che regione provieni?")
    .drop("Provincia di provenienza")
    .drop("In che regione lavori/studi?")
    .drop("Provincia del luogo di lavoro/studio")
    .drop("Invalidità")
    .drop("Tipo di residenza")
    .drop("Numero di persone con cui convivi")
    .drop("Entrate Familiari Mensili Nette")
    .drop("Entrate Personali Mensili Nette ")
)

df.show(5)

+-----------------------------------------+-----------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+---------------------+------------------------------------+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+---------------------------------------------------------------------------------------------

In [76]:
#Lettura file json contenente gli score
with open(r"C:\Users\palaz\OneDrive\Desktop\University\UNIPA 2.0\I ANNO\II semestre\Data Processing\BDM - Big Data Management\Progetto 2\BigDataProject\data\questions.json", "r", encoding="utf-8") as scores_dir:
    scores = json.load(scores_dir)

#Creo dataframe contenente gli score di ogni risposta
question_idx = []           #Indici delle domande prese dal json - sfrutto per nominare l'header del dataframe nuovo
for i in scores:
    question_idx.append(i)

questions = {scores[q]["question_text"]:q for q in scores.keys()}

data_scores = []                #Lista contenente le liste di score per ogni colonna
for col in df.columns:    
    idx = questions[col]

    for row in range(53):
        x = df.at(row,col)      #Accedo ad ogni singolo elemento della colonna
        x_score = []            #Lista contenente gli score di quella colonna
        for i in range(5):
            if x == scores[idx]["answers"][i]["answer_text"]:
                y = scores[idx]["answers"][i]["answer_text"]["answer_score"] * scores[idx]["question_score"]
                x_score.append(y)
                break
    data_scores.append(x_score)






df1 = spark.createDataFrame( data = data_scores, schema = question_idx)
df1.show()




AttributeError: 'DataFrame' object has no attribute 'at'